In [22]:
!pip install -q chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 4.0 MB/s eta 0:00:

# Import

In [34]:
import pandas as pd
import numpy as np
#Timer
import time
from timeit import default_timer as timer

#embed
import random
import torch
from sentence_transformers import util, SentenceTransformer

# Chromadb
import chromadb
from chromadb.config import Settings

# Read data

In [9]:
data_1k = pd.read_csv("/content/text_chunks_and_embeddings_1k_df.csv")
data_1k.shape

(1680, 6)

In [10]:
 data_1k.head(2)

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,-39,Human Nutrition: 2020 Edition UNIVERSITY OF HA...,308,42,77.0,[ 6.74242675e-02 9.02281404e-02 -5.09548886e-...
1,-38,Human Nutrition: 2020 Edition by University of...,210,30,52.5,[ 5.52156419e-02 5.92139773e-02 -1.66167244e-...


In [11]:
data_10k = pd.read_csv("/content/text_chunks_and_embeddings_10k_df.csv")
data_10k.shape

(11841, 6)

In [12]:
data_10k.head(2)

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,-41,Human Nutrition: 2020 Edition,29,4,7.25,[ 4.41241562e-02 9.24154669e-02 3.28843738e-...
1,-39,Human Nutrition: 2020 Edition UNIVERSITY OF HA...,308,42,77.00,[ 6.74242601e-02 9.02281925e-02 -5.09549258e-...


In [13]:
data_100k_1 = pd.read_csv("/content/text_chunks_and_embeddings_100k_1_df.csv")
data_100k_2 = pd.read_csv("/content/text_chunks_and_embeddings_100k_2_df.csv")
# Merge (stack rows)
data_100k = pd.concat([data_100k_1, data_100k_2], ignore_index=True)
data_100k.shape

(116419, 6)

In [14]:
data_100k.head(2)

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,-41,Human Nutrition,15,2,3.75,"tensor([ 5.4477e-02, 3.9437e-02, 5.7889e-03,..."
1,-41,: 2020,6,2,1.50,"tensor([-3.1906e-03, 8.9563e-02, -2.3828e-02,..."


# Processing

In [16]:
#1k, 10k
data_1k["embedding"] = data_1k["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))
data_10k["embedding"] = data_10k["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))

#100k
data_100k["embedding"] = data_100k["embedding"].apply(lambda x: np.fromstring(x.split("[")[1].split("]")[0][:-1].replace(",", ""), sep=" "))

In [19]:
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2",
                                      device="cpu")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
query = "macronutrients functions"
query_embedding = embedding_model.encode(query, convert_to_tensor=True)

In [23]:
# embeddings_np = np.vstack(data_1k["embedding"].tolist())
# embeddings_tensor = torch.tensor(embeddings_np, dtype=torch.float32)
# embeddings_tensor = embeddings_tensor.to(query_embedding.device)

In [40]:
def compare_dot_chromadb(df):
  embeddings_np = np.vstack(df["embedding"].tolist())
  embeddings_tensor = torch.tensor(embeddings_np, dtype=torch.float32)
  embeddings_tensor = embeddings_tensor.to(query_embedding.device)

  # dot product
  start = timer()
  dot_scores = util.dot_score(query_embedding, embeddings_tensor)[0]
  top_dot = torch.topk(dot_scores, k=5)
  end = timer()
  dot_time = end - start

  # chromadb
  client = chromadb.PersistentClient(path="db/")
  collection = client.get_or_create_collection(name="comparison")

  batch_size = 5000  # keep it below ChromaDB's max (5461)

  documents = df["sentence_chunk"].tolist()
  embeddings = df["embedding"].tolist()
  ids = [f"id_{i}" for i in range(len(df))]

  for i in range(0, len(ids), batch_size):
      collection.add(
          documents=documents[i:i + batch_size],
          embeddings=embeddings[i:i + batch_size],
          ids=ids[i:i + batch_size]
      )

  # Query
  start = timer()
  results = collection.query(query_embeddings=[query_embedding.tolist()], n_results=5)
  end = timer()
  chroma_time = end - start

  # IDs from dot product (assuming df["text"] is used)
  top_dot_indices = top_dot.indices.cpu().numpy()
  top_dot_texts = set(df.iloc[top_dot_indices]["sentence_chunk"])

  chroma_texts = set(results["documents"][0])
  accuracy = len(top_dot_texts.intersection(chroma_texts)) / 5

  # return df
  comparison_df = pd.DataFrame([
      {
          "Method": "Dot Product",
          "Num Embeddings": len(df),
          "Query Time (s)": dot_time,
          "Retrieval Accuracy": 1.0,
          "Notes": "Exact match"
      },
      {
          "Method": "ChromaDB",
          "Num Embeddings": len(df),
          "Query Time (s)": chroma_time,
          "Retrieval Accuracy": accuracy,
          "Notes": "Approximate match (FAISS/HNSW)"
      }
  ])
  return comparison_df

In [41]:
compare_dot_chromadb(data_1k)

,Method,Num Embeddings,Query Time (s),Retrieval Accuracy,Notes
0,Dot Product,1680,0.000671,1.0,Exact match
1,ChromaDB,1680,0.002955,1.0,Approximate match (FAISS/HNSW)


In [42]:
compare_dot_chromadb(data_10k)

,Method,Num Embeddings,Query Time (s),Retrieval Accuracy,Notes
0,Dot Product,11841,0.003239,1.0,Exact match
1,ChromaDB,11841,0.003432,0.0,Approximate match (FAISS/HNSW)


In [43]:
compare_dot_chromadb(data_100k)

,Method,Num Embeddings,Query Time (s),Retrieval Accuracy,Notes
0,Dot Product,116419,0.050436,1.0,Exact match
1,ChromaDB,116419,0.004063,0.0,Approximate match (FAISS/HNSW)


# Differences between using dot product and chromadb (knowledge)
## 🔁 Similarity Search: Dot Product vs. ChromaDB

You want to find which texts (from a dataset) are most **similar** to a query using **vector embeddings**. Here's a detailed comparison:

---

### 🧠 Embedding Similarity: Manual Dot Product vs ChromaDB

| Aspect | **Manual Dot Product (PyTorch)** | **ChromaDB (Vector DB)** |
|--------|----------------------------------|---------------------------|
| **Method** | Compute dot product between query and all dataset embeddings using `util.dot_score()` | Store embeddings in a vector DB and use `.query()` with the query embedding |
| **Speed (small data)** | Very fast for small datasets (e.g., <10k vectors) | Slight overhead from DB operations; may be slightly slower |
| **Speed (large data)** | Slows down with large datasets (linear scan over all vectors) | Fast search with Approximate Nearest Neighbor (ANN) indexes |
| **Memory** | Needs to load all embeddings into RAM as a tensor | Stores on disk, optimized for memory and I/O |
| **Scalability** | Not scalable — memory/time grow linearly | Built for scale — constant/sublinear time for large datasets |
| **Query capabilities** | Basic similarity math only (dot product, cosine) | Supports similarity + metadata filters |
| **Result Format** | `torch.topk()` gives top-k scores and indices | Dictionary with `'ids'`, `'documents'`, `'distances'` |
| **Flexibility** | Fully customizable similarity logic | Less control over internal similarity metric (ANN) |

---

### 🧪 When to Use Which?

| Situation | Recommendation |
|-----------|----------------|
| Small dataset (<5k vectors) | ✅ Dot product |
| Need fast prototyping | ✅ Dot product |
| Doing research with custom similarity logic | ✅ Dot product |
| Need persistent index across sessions | ✅ ChromaDB |
| Scaling to 100k+ vectors | ✅ ChromaDB |
| Need metadata filtering with similarity | ✅ ChromaDB |

---

### 🔧 Code Comparison

#### 🧮 Dot Product Example:
```python
dot_scores = util.dot_score(query_embedding, embeddings)[0]
top_results = torch.topk(dot_scores, k=5)

# Test

In [44]:
# # dot product
# start = timer()
# dot_scores = util.dot_score(query_embedding, embeddings_tensor)[0]
# top_dot = torch.topk(dot_scores, k=5)
# end = timer()
# dot_time = end - start

In [45]:
# client = chromadb.PersistentClient(path="db/")
# collection = client.get_or_create_collection(name="comparison")

# batch_size = 5000  # keep it below ChromaDB's max (5461)

# documents = data_1k["sentence_chunk"].tolist()
# embeddings = data_1k["embedding"].tolist()
# ids = [f"id_{i}" for i in range(len(data_1k))]

# for i in range(0, len(ids), batch_size):
#     collection.add(
#         documents=documents[i:i + batch_size],
#         embeddings=embeddings[i:i + batch_size],
#         ids=ids[i:i + batch_size]
#     )

# # Query
# start = timer()
# results = collection.query(query_embeddings=[query_embedding.tolist()], n_results=5)
# end = timer()
# chroma_time = end - start

In [46]:
# # IDs from dot product (assuming df["text"] is used)
# top_dot_indices = top_dot.indices.cpu().numpy()
# top_dot_texts = set(data_1k.iloc[top_dot_indices]["sentence_chunk"])

# chroma_texts = set(results["documents"][0])
# accuracy = len(top_dot_texts.intersection(chroma_texts)) / 5


In [47]:
# comparison_df = pd.DataFrame([
#     {
#         "Method": "Dot Product",
#         "Num Embeddings": len(data_1k),
#         "Query Time (s)": dot_time,
#         "Retrieval Accuracy": 1.0,
#         "Notes": "Exact match"
#     },
#     {
#         "Method": "ChromaDB",
#         "Num Embeddings": len(data_1k),
#         "Query Time (s)": chroma_time,
#         "Retrieval Accuracy": accuracy,
#         "Notes": "Approximate match (FAISS/HNSW)"
#     }
# ])
# comparison_df